In [2]:
from sage.matrix.operation_table import OperationTable
from sage.arith.functions import LCM_list
import time

In [24]:
G1 = DihedralGroup(10)
G2 = CyclicPermutationGroup(16)
G3 = AlternatingGroup(5)
G4 = SL(2, Integers(3))
groups = {
    G1 : "D_10",
    G2 : "Z_16",
    G3 : "A_5",
    G4 : "SL(2, Z3)"
}

with open("task1.txt", "w") as f:
    for group in groups.keys():
        print(f"Group: {groups[group]}", file = f)
        print(OperationTable(group, operator.mul), file = f)
        print(f"Group order: {group.cardinality()}", file = f)
        print(f"The group is abelian: {group.is_abelian()}", file = f)
        print("\n", file = f)
    print(f"D12 is isomorphic to SL(2, Z3): {G4.is_isomorphic(DihedralGroup(12))}", file = f)

In [3]:
def length_amount(partition):
    """The amount of cycles with length <number>, 
    where <number> is an element from <partition>.
    """
    counter = 0
    current_length = partition[0]
    for length in partition:
        if length != current_length:
            current_length = length
            yield counter
            counter = 1
        else:
            counter += 1
    yield counter

factorials = {0: 1}
res = 1
for i in range(1, 101):
    res *= i
    factorials[i] = res
    
def number(partition, n):
    """The number of permutations from S_n with <partition> cycle type.
    """
    global factorials
    res = 1
    for length in partition:
        res *= factorials[n] / factorials[n - length] / length
        n -= length
    for amount in length_amount(partition):
        if amount == 1:
            pass
        else:
            res /= factorials[amount]
    return res

def order_amount(n):
    """The number of elements of every possible order from S_n.
    """
    order_amount = {}
    unique_partitions = Partitions(n)
    for partition in unique_partitions:
        ord_ = LCM_list(partition)
        if ord_ not in order_amount:
            order_amount[ord_] = number(partition, n)
        else:
            order_amount[ord_] += number(partition, n)
    keys = list(order_amount.keys())
    keys.sort()
    return {i:order_amount[i] for i in keys}

start = time.time()
n = 100
with open(f"orders_s{n}.txt", "w") as f:
    counter = 0
    for order, amount in order_amount(n).items():
        print(f"Order: {order}, number of elements: {amount}", file = f)
        counter += amount
print(f"Elapsed time: {time.time() - start}")
print(counter == factorial(n))


Elapsed time: 6179.146168231964
True


In [88]:
def is_even(partition):
    """Defines whether a permutation with the cycle type
    represented by <partition> is even.
    """
    counter = 0
    for length in partition:
        if length % 2 == 1:
            pass
        else:
            counter += 1
    if counter % 2 == 0:
        return True
    else:
        return False

def order_amount(n):
    """The number of elements of every possible order from A_n.
    """
    order_amount = {}
    unique_partitions = Partitions(n)
    for partition in unique_partitions:
        if not is_even(partition):
            continue
        else:
            ord_ = LCM_list(partition)
            if ord_ not in order_amount:
                order_amount[ord_] = number(partition, n)
            else:
                order_amount[ord_] += number(partition, n)
    keys = list(order_amount.keys())
    keys.sort()
    return {i:order_amount[i] for i in keys}

n = 100
start = time.time()
with open(f"orders_a{n}.txt", "w") as f:
    counter = 0
    for order, amount in order_amount(n).items():
        print(f"Order: {order}, number of elements: {amount}", file = f)
        counter += amount
print(f"Elapsed time: {time.time() - start}")
print(counter == factorial(n) / 2)


Elapsed time: 4547.163512706757
True


In [40]:
def exists_naive(k, n):
    """Defines whether there exists an element of order <k> in S_n.
    Complexity : O(exp(pi * sqrt(2 * n / 3))).
    """
    if factorial(n) % k != 0:
        return False, None
    partitions = Partitions(n)
    for partition in partitions:
        if LCM_list(partition) != k:
            continue
        else:
            return True, partition
    return False, None

def correctness(n):
    """Checks the existence of an element of order <k> in S_n
    for k from {1, ..., n!}
    """
    start = time.time()
    with open(f"orders_s{n}_naive.txt", "w") as f:
        for k in range(1, factorial(n) + 1):
                res = exists_naive(k, n)
                if not res[0]:
                    print(f"k = {k}, exists: False", file = f)
                else:
                    print(f"k = {k}, exists with the cycle type : {res[1]}", file = f)
    print(f"Elapsed time: {time.time() - start}")
correctness(10)

Elapsed time: 5.34474515914917


In [3]:
def exists(k, n):
    """Defines whether there exists an element of order <k> in S_n.
    Complexity : O(2^n).
    """
    if factorial(n) % k != 0:
        return False, None
    divs = iter(divisors(k))
    possible_lengths = []
    for length in divs:
        if sum(possible_lengths) > n:
            break
        elif length > n:
            break
        else:
            possible_lengths += [length]
    candidates = iter(Subsets(possible_lengths))
    for candidate in candidates:
        if sum(candidate) > n:
            continue
        if LCM_list(candidate) != k:
            continue
        else:
            return True, candidate
    return False, None

def correctness(n):
    """Checks the existence of an element of order <k> in S_n
    for k from {1, ..., n!}
    """
    start = time.time()
    with open(f"orders_s{n}.txt", "w") as f:
        print("Cycles with length 1 are omitted in cycle types.", file = f)
        print(f"n = {n}", file = f)
        for k in range(1, factorial(n) + 1):
            res = exists(k, n)
            if not res[0]:
                print(f"k = {k}, exists: False", file = f)
            else:
                print(f"k = {k}, exists with the cycle type : {res[1]}", file = f)
    print(f"Elapsed time: {time.time() - start}")
correctness(10)

Elapsed time: 5.245164632797241


In [34]:
with open("check_orders_s10.txt", "w") as f:
    counter = 0
    for order, amount in order_amount(10).items():
        print(f"Order: {order}, number of elements: {amount}", file = f)
        counter += amount